In [318]:
import pandas as pd
from google.cloud import bigquery
# プロジェクトIDの指定
project_id = 'inunaki'
# authが終わっていたら普通にクライアント認証は通る
client = bigquery.Client(project=project_id)

In [319]:
query = """
SELECT DATE(year, month, dt) AS dt, sync_dttm, customer_id, coupon_id, coupon_name 
FROM `inunaki.dwd_action_0001.dwd_karte_user_use_coupon` 
WHERE DATE(year, month, dt) BETWEEN "2023-03-06" AND "2023-04-30"
"""

#データフレームに変換
df_cupon_use = client.query(query).to_dataframe()

df_cupon_use["salesday"] = pd.to_datetime(df_cupon_use["dt"])

In [320]:
df_cupon_use.head()

,dt,sync_dttm,customer_id,coupon_id,coupon_name,salesday
0,2023-03-06,2023-03-06 17:39:10.168,a9c1c0cd09705da3f8777a11abe9c041,59,腸までとどくN-1プレーンヨーグルト,2023-03-06
1,2023-03-06,2023-03-06 18:36:19.392,470e4239df728717e08da7e0eb2ed503,59,腸までとどくN-1プレーンヨーグルト,2023-03-06
2,2023-03-06,2023-03-06 17:11:11.810,49090e72488f91f425b2bf729f4cfa6f,59,腸までとどくN-1プレーンヨーグルト,2023-03-06
3,2023-03-06,2023-03-06 17:45:39.659,f16fbb7a9635ab5b6b56605f3778d4f8,59,腸までとどくN-1プレーンヨーグルト,2023-03-06
4,2023-03-06,2023-03-06 14:34:31.601,58f54ef655feafbefd4abb6a2c5d2e0c,59,腸までとどくN-1プレーンヨーグルト,2023-03-06


In [321]:
query = """
SELECT DISTINCT coupon_id, coupon_name
FROM `inunaki.dwd_action_0001.dwd_karte_user_use_coupon` 
WHERE DATE(year, month, dt) BETWEEN "2023-03-06" AND "2023-04-30"
"""
df_coupon_id = client.query(query).to_dataframe()

In [322]:
df_coupon_id.head()

,coupon_id,coupon_name
0,95,おかめ仕立てうまかたれ付45g×3
1,96,ブルガリアヨーグルトLB81プレーン
2,97,みどり_豆乳_成分無調整1000ml
3,98,ラーマ_バターの風味
4,99,牛焼肉重


In [323]:
len(df_coupon_id)

55

In [324]:
# あとで使うように残しておく
df_cupon_sengen = df_cupon_use.copy()

In [325]:
query = """
WITH base AS(
  SELECT  salesday, COALESCE(customercd_new, posid.customercd) AS customercd, jan, SUM(salesamount)AS sales_amt, SUM(salescnt)AS cnt
  FROM `inunaki.dwd_cdp_0001.dwd_cdp_idpos_detail` posid 
  LEFT JOIN `inunaki.dim_common_0001.dim_crm_customer_change_history` ii
  ON posid.customercd = ii.customercd
  WHERE salesday BETWEEN "2023-03-06" AND "2023-04-30"
  AND storecd IN(8,57,67,157,158,282,472,473)
  AND jan IN(
  '00000004901516013682',
 '00000002548098000000',
 '00000002553756000000',
 '00000002553757000000',
 '00000002548100000000',
 '00000004562496240194',
 '00000004903050502806',
 '00000004901990548748',
 '00000004902720150972',
 '00000004901516004833',
 '00000004538067535914',
 '00000004538067531893',
 '00000004951595708092',
 '00000002553817000000',
 '00000002550644000000',
 '00000006970747940492',
 '00000006970747940812',
 '00000004904681971153',
 '00000004967777004432',
 '00000004902410112280',
 '00000004902410112297',
 '00000004978006910103',
 '00000004962035612466',
 '00000004522646718102',
 '00000004522646773323',
 '00000004522646813814',
 '00000004901160010044',
 '00000004901160010174',
 '00000004902705011625',
 '00000004901516027573',
 '00000004902590127852',
 '00000002556098000000',
 '00000002554449000000',
 '00000002537850000000',
 '00000002553766000000',
 '00000002553765000000',
 '00000004901516005472',
 '00000004902585103083',
 '00000006970747940645',
 '00000006970747940317',
 '00000006970747940300',
 '00000004902560020619',
 '00000004522646773347',
 '00000004580575373427',
 '00000004580575373410',
 '00000004967777004463',
 '00000004902115053727',
 '00000004522646813784',
 '00000006970747940218')
  AND posid.customercd <> "0"
  GROUP BY salesday, customercd, jan
 )
 SELECT　salesday, customercd, base.jan, sales_amt, cnt, janname
 FROM base
JOIN `inunaki.dim_common_0001.dim_product` a
ON a.jan = base.jan
"""
#データフレームに変換
df_jan = client.query(query).to_dataframe()

df_jan["jan2"] = df_jan["jan"].astype(int)
df_jan["salesday"] = pd.to_datetime(df_jan["salesday"])

In [326]:
df_jan.head()

,salesday,customercd,jan,sales_amt,cnt,janname,jan2
0,2023-04-09,651ce7fa0f64503aad03f6b220ae4a0e,00000004904681971153,199,1,くっつかないホイル 25ｃｍ×15ｍ,4904681971153
1,2023-04-09,403453d5f86897b4237c3a5504fc6dd0,00000004904681971153,398,2,くっつかないホイル 25ｃｍ×15ｍ,4904681971153
2,2023-04-09,e618bf1035d621319ffd48b43027b4ba,00000004904681971153,199,1,くっつかないホイル 25ｃｍ×15ｍ,4904681971153
3,2023-04-09,adb894a92804ed0dd22672184a244c4b,00000004904681971153,199,1,くっつかないホイル 25ｃｍ×15ｍ,4904681971153
4,2023-04-09,7487036701e8dd0975729efb5ca5e691,00000004904681971153,199,1,くっつかないホイル 25ｃｍ×15ｍ,4904681971153


In [327]:
## クーポンタブを見た人
query = """
SELECT DATE(year, month, dt) AS dt, sync_dttm, customer_id ,view_title
FROM `inunaki.dwd_action_0001.dwd_karte_user_view_couponlist` 
WHERE DATE(year, month, dt) BETWEEN "2023-03-06" AND "2023-04-30"
"""


df_c_view = client.query(query).to_dataframe()

# df_c_view["sync_date"] = pd.to_datetime(df_c_view["sync_date"])
# df_c_view["salesday"] = df_c_view["sync_date"].dt.strftime("%Y-%m-%d")
df_c_view["salesday"] = pd.to_datetime(df_c_view["dt"])

In [328]:
df_c_view.head()

,dt,sync_dttm,customer_id,view_title,salesday
0,2023-04-18,2023-04-18 06:54:10.233,0011245f6d31e007424bfbaa36be458e,クーポン一覧,2023-04-18
1,2023-04-27,2023-04-27 08:45:10.685,0011245f6d31e007424bfbaa36be458e,クーポン一覧,2023-04-27
2,2023-04-27,2023-04-27 08:46:02.794,0011245f6d31e007424bfbaa36be458e,クーポン一覧,2023-04-27
3,2023-04-27,2023-04-27 08:44:47.402,0011245f6d31e007424bfbaa36be458e,クーポン一覧,2023-04-27
4,2023-04-27,2023-04-27 08:43:45.639,0011245f6d31e007424bfbaa36be458e,クーポン一覧,2023-04-27


In [329]:
# df_c_view2 = df_c_view.copy()

In [330]:
# ホームを見た人
query = """
SELECT sync_dttm, DATE(year, month, dt) AS dt, customer_id, view_name
FROM `inunaki.dwd_action_0001.dwd_karte_supay_accesshome` view_data 
WHERE DATE(year, month, dt) BETWEEN "2023-03-06" AND "2023-04-30"
"""

df_home_view = client.query(query).to_dataframe()
df_home_view["salesday"] = pd.to_datetime(df_home_view["dt"])



query  = """
SELECT customercd_md5, a.customercd
FROM `inunaki.dim_crm_0001.dim_crm_customer_md5` a
JOIN `inunaki.dst_mkscience.app_coupon_taisho_3m_4m` b
ON CAST(a.customercd AS INT64) = CAST(b.customercd AS INT64)
"""
df_id_hashu = client.query(query).to_dataframe()
df_id_hashu = df_id_hashu.rename(columns={"customercd_md5":"customer_id"})
# ホーム画面を見た人を対象者だけに絞る
df_home_view = df_home_view.merge(df_id_hashu,on="customer_id")

In [331]:
df_home_view.head()

,sync_dttm,dt,customer_id,view_name,salesday,customercd
0,2023-03-06 13:07:13.705,2023-03-06,5b758405250fd725302f6e4e0f5b020e,Home,2023-03-06,2970003066045
1,2023-03-17 16:25:01.092,2023-03-17,5b758405250fd725302f6e4e0f5b020e,Home,2023-03-17,2970003066045
2,2023-03-16 18:53:37.032,2023-03-16,5b758405250fd725302f6e4e0f5b020e,Home,2023-03-16,2970003066045
3,2023-04-15 12:02:11.792,2023-04-15,5b758405250fd725302f6e4e0f5b020e,Home,2023-04-15,2970003066045
4,2023-04-23 02:48:51.873,2023-04-23,5b758405250fd725302f6e4e0f5b020e,Home,2023-04-23,2970003066045


In [332]:
# query = """
# SELECT *
# FROM `inunaki.dst_workspace.app_cupon_jan`
# WHERE (haisin_tuki = "2023-03-01") OR (haisin_tuki = "2023-04-01")
# """
# # #データフレームに変換
# df_taisho_id = client.query(query).to_dataframe()
df_taisho_id = pd.read_csv("data/クーポン情報_3月4月.csv")
df_taisho_id["jan"] = df_taisho_id["jan"].astype(str)

In [333]:
# df_taisho_id.to_csv("クーポン情報_3月4月.csv",index=False,encoding="utf_8_sig")

In [334]:
# パワープレーでの変更
df_tmp = df_taisho_id.query("coupon_name=='かしわおにぎり_2個入り'")
df_tmp = df_tmp.drop("jan",axis=1)
df_tmp["jan"] = 2553756000000

df_tmp2 = df_taisho_id.query("coupon_name=='おかめ仕立てうまかたれ付45g×3'")
df_tmp2 = df_tmp2.drop("jan",axis=1)
df_tmp2["jan"] = 4901160010174

df_taisho_id = df_taisho_id.query("jan!='2548098000000'").query("jan!='4901160010044'")
df_taisho_id = pd.concat([df_taisho_id, df_tmp, df_tmp2])

In [360]:
df_taisho_id = df_taisho_id.drop_duplicates()

In [335]:
# janをつける ## coupon_name
df_cupon_use = df_cupon_use.merge(df_taisho_id[["coupon_name","jan"]].drop_duplicates(),on="coupon_name",how="left")

In [336]:
df_cupon_use[df_cupon_use["jan"].isnull()].sort_values("jan")

df_cupon_use = df_cupon_use.dropna()

df_cupon_use["jan2"] = df_cupon_use["jan"].astype(int)

In [337]:
# 共通クーポンのみに絞る
df_taishoh_id_kyoutu = df_taisho_id.query("kubun=='共通'")[["coupon_name","kubun"]].drop_duplicates()
df_cupon_use_kyotu = df_cupon_use.merge(df_taishoh_id_kyoutu,on="coupon_name")

In [338]:
# 3月の結果 df_cupon_use
df_date2 = pd.DataFrame()
for i in df_cupon_use_kyotu.query("salesday>=20230306&salesday<=20230331")["customer_id"].unique():
    # 日付を作る
    df_date = pd.DataFrame({"salesday":pd.date_range(start="2023-03-06",end="2023-03-31")})
    df_date["a"] = 1
    # クーポン対象商品を出す
    df_a = df_cupon_use_kyotu.query("customer_id==@i").reset_index().drop("index",axis=1)[["salesday","coupon_name","jan2","customer_id"]]\
    .rename(columns={"salesday":"start_date"})
    df_a["a"] = 1
    # 日毎に作る
    df_date = df_date.merge(df_a,on="a")
    del df_date["a"]
    df_date["flg"] = df_date.apply(lambda x:1 if x["start_date"]<=x["salesday"] else 0, axis=1)
    df_date = df_date.query("flg==1")
    df_date2 = pd.concat([df_date2,df_date])

df_result_3m = df_jan.merge(df_date2.rename(columns={"customer_id":"customercd"}),on=["salesday","customercd","jan2"])
# df_result_3m = df_jan.merge(df_date2.rename(columns={"jan":"jan2","user_id":"customercd2"}),on=["salesday","customercd2","jan2"])
# 重複がないかは確認
df_result_3m = df_result_3m.groupby(["customercd","coupon_name"]).head(1)

In [339]:
# 4月の結果 df_cupon_use_kyotu
df_date2 = pd.DataFrame()
for i in df_cupon_use_kyotu.query("salesday>=20230401&salesday<=20230430")["customer_id"].unique():
    # 日付を作る
    df_date = pd.DataFrame({"salesday":pd.date_range(start="2023-04-01",end="2023-04-30")})
    df_date["a"] = 1
    # クーポン対象商品を出す
    df_a = df_cupon_use_kyotu.query("customer_id==@i").reset_index().drop("index",axis=1)[["salesday","coupon_name","jan2","customer_id"]]\
    .rename(columns={"salesday":"start_date"})
    df_a["a"] = 1
    # 日毎に作る
    df_date = df_date.merge(df_a,on="a")
    del df_date["a"]
    df_date["flg"] = df_date.apply(lambda x:1 if x["start_date"]<=x["salesday"] else 0, axis=1)
    df_date = df_date.query("flg==1")
    df_date2 = pd.concat([df_date2,df_date])

df_result_4m = df_jan.merge(df_date2.rename(columns={"customer_id":"customercd"}),on=["salesday","customercd","jan2"])

# 重複がないかは確認
df_result_4m = df_result_4m.groupby(["customercd","coupon_name"]).head(1)

In [340]:
df_result_4m["coupon_name"].nunique()

17

In [341]:
# 重複を削除
df_cupon_use = df_cupon_use.groupby(["coupon_name","customer_id"]).head(1)
df_cupon_use_kyotu = df_cupon_use_kyotu.groupby(["coupon_name","customer_id"]).head(1)

### 3月の共通クーポンのみ

In [342]:
# 3月の結果について（共通のみ）
# 利用者
df_tmp_riyou_3m = df_result_3m.groupby(["salesday"])\
.agg({"flg":"sum","customercd":"nunique"}).reset_index().rename(columns={"flg":"クーポン利用人数","customercd":"クーポン利用人数UU"})
# 宣言人数
df_tmp_sengen_3m = df_cupon_use_kyotu.query("salesday>=20230306&salesday<=20230331").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"宣言人数","customer_id":"宣言人数UU"})
# クーポンタブ閲覧
df_tmp_eturan_3m = df_c_view.query("salesday>=20230306&salesday<=20230331").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"クーポンタブ閲覧人数","customer_id":"クーポンタブ閲覧UU"})

# 累積宣言UU
df_sengen_rui_3m = df_cupon_use_kyotu.sort_values("salesday").query("salesday>=20230306&salesday<=20230331")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積宣言UU"})
df_sengen_rui_3m["累積宣言UU"]= df_sengen_rui_3m["累積宣言UU"].cumsum()

# 累積クーポンタブUU
df_eturan_rui_3m = df_c_view.sort_values("salesday").query("salesday>=20230306&salesday<=20230331")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積クーポン閲覧UU"})
df_eturan_rui_3m["累積クーポン閲覧UU"]= df_eturan_rui_3m["累積クーポン閲覧UU"].cumsum()

# 累積購入クーポンUU
df_riyo_rui3m = df_result_3m.sort_values("salesday").query("salesday>=20230306&salesday<=20230331")\
.groupby(["customercd"]).head(1).groupby(["salesday"]).agg({"customercd":"count"}).reset_index().rename(columns={"customercd":"累積クーポン利用UU"})
df_riyo_rui3m["累積クーポン利用UU"]= df_riyo_rui3m["累積クーポン利用UU"].cumsum()

home_3m_kyoutu = df_home_view.query("salesday>=20230306&salesday<=20230331")["customer_id"].nunique()

# 結果を表示（共通のみ）
df_result2 = df_tmp_riyou_3m.merge(df_tmp_sengen_3m,on="salesday",how="outer")
df_result2 = df_result2.merge(df_tmp_eturan_3m, on="salesday",how="outer")
df_result2 = df_result2.merge(df_sengen_rui_3m, on="salesday",how="outer")
df_result2 = df_result2.merge(df_eturan_rui_3m, on="salesday",how="outer")
df_result2 = df_result2.merge(df_riyo_rui3m, on="salesday",how="outer")
df_result2 = df_result2.rename(columns={"salesday":"日付"})
df_result2["アプリ起動UU"] = home_3m_kyoutu

df_result2.query("日付=='20230331'")[["アプリ起動UU","累積クーポン閲覧UU","累積宣言UU","累積クーポン利用UU"]]

,アプリ起動UU,累積クーポン閲覧UU,累積宣言UU,累積クーポン利用UU
25,8718,3964,482,244


In [343]:
# 4月の結果について（共通のみ）
# 利用者
df_tmp_riyou_4m = df_result_4m.groupby(["salesday"])\
.agg({"flg":"sum","customercd":"nunique"}).reset_index().rename(columns={"flg":"クーポン利用人数","customercd":"クーポン利用人数UU"})
# 宣言人数
df_tmp_sengen_3m = df_cupon_use_kyotu.query("salesday>=20230401&salesday<=20230430").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"宣言人数","customer_id":"宣言人数UU"})
# クーポンタブ閲覧
df_tmp_eturan_3m = df_c_view.query("salesday>=20230401&salesday<=20230430").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"クーポンタブ閲覧人数","customer_id":"クーポンタブ閲覧UU"})

# 累積宣言UU
df_sengen_rui_3m = df_cupon_use_kyotu.sort_values("salesday").query("salesday>=20230401&salesday<=20230430")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積宣言UU"})
df_sengen_rui_3m["累積宣言UU"]= df_sengen_rui_3m["累積宣言UU"].cumsum()

# 累積クーポンタブUU
df_eturan_rui_3m = df_c_view.sort_values("salesday").query("salesday>=20230401&salesday<=20230430")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積クーポン閲覧UU"})
df_eturan_rui_3m["累積クーポン閲覧UU"]= df_eturan_rui_3m["累積クーポン閲覧UU"].cumsum()

# 累積購入クーポンUU
df_riyo_rui3m = df_result_4m.sort_values("salesday").query("salesday>=20230401&salesday<=20230430")\
.groupby(["customercd"]).head(1).groupby(["salesday"]).agg({"customercd":"count"}).reset_index().rename(columns={"customercd":"累積クーポン利用UU"})
df_riyo_rui3m["累積クーポン利用UU"]= df_riyo_rui3m["累積クーポン利用UU"].cumsum()

home_3m_kyoutu = df_home_view.query("salesday>=20230401&salesday<=20230430")["customer_id"].nunique()

# 結果を表示（共通のみ）
df_result2_4m = df_tmp_riyou_3m.merge(df_tmp_sengen_3m,on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_tmp_eturan_3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_sengen_rui_3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_eturan_rui_3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_riyo_rui3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.rename(columns={"salesday":"日付"})
df_result2_4m["アプリ起動UU"] = home_3m_kyoutu

df_result2_4m.query("日付=='20230430'")[["アプリ起動UU","累積クーポン閲覧UU","累積宣言UU","累積クーポン利用UU"]]

,アプリ起動UU,累積クーポン閲覧UU,累積宣言UU,累積クーポン利用UU
55,8797,3534.0,467.0,271.0


### ランクでの違い

In [344]:
# 対象顧客の情報を出す
df_taisho_id_rank = pd.read_csv("data/taisho_id.csv").rename(columns={"顧客ID":"customercd"})
df_taisho_id_rank["customercd"] = df_taisho_id_rank["customercd"].astype(str)
df_taisho_id_rank = df_taisho_id_rank.query("ランク!='1月購買実績なし'")
df_taisho_id_rank = df_id_hashu.merge(df_taisho_id_rank,on="customercd")

In [345]:
a = df_taisho_id_rank.merge(df_cupon_use_kyotu, on="customer_id").groupby(["ランク"]).agg({"customercd":"count"}).rename(columns={"customercd":"宣言人数"})
ab = df_taisho_id_rank.groupby(["ランク"]).agg({"customercd":"count"}).rename(columns={"customercd":"全体数"})
a = a.merge(ab,on="ランク")
a["比率"] = round(a["宣言人数"] / a["全体数"], 3)
a.reindex(index=["H","M","L","E"])

,宣言人数,全体数,比率
ランク,,,
H,353,668,0.528
M,660,1597,0.413
L,657,2181,0.301
E,645,5182,0.124


In [346]:
df_hm = df_taisho_id_rank.query("ランク=='H'|ランク=='M'")[["customer_id"]]

In [347]:
# 共通クーポンのみに絞る
df_taishoh_id_kyoutu = df_taisho_id.query("kubun=='PB'")[["coupon_name","kubun"]].drop_duplicates()
df_cupon_use_kyotu = df_cupon_use.merge(df_taishoh_id_kyoutu,on="coupon_name")


# 3月の結果 df_cupon_use
df_date2 = pd.DataFrame()
for i in df_cupon_use_kyotu.query("salesday>=20230306&salesday<=20230331")["customer_id"].unique():
    # 日付を作る
    df_date = pd.DataFrame({"salesday":pd.date_range(start="2023-03-06",end="2023-03-31")})
    df_date["a"] = 1
    # クーポン対象商品を出す
    df_a = df_cupon_use_kyotu.query("customer_id==@i").reset_index().drop("index",axis=1)[["salesday","coupon_name","jan2","customer_id"]]\
    .rename(columns={"salesday":"start_date"})
    df_a["a"] = 1
    # 日毎に作る
    df_date = df_date.merge(df_a,on="a")
    del df_date["a"]
    df_date["flg"] = df_date.apply(lambda x:1 if x["start_date"]<=x["salesday"] else 0, axis=1)
    df_date = df_date.query("flg==1")
    df_date2 = pd.concat([df_date2,df_date])

df_result_3m = df_jan.merge(df_date2.rename(columns={"customer_id":"customercd"}),on=["salesday","customercd","jan2"])
# df_result_3m = df_jan.merge(df_date2.rename(columns={"jan":"jan2","user_id":"customercd2"}),on=["salesday","customercd2","jan2"])
# 重複がないかは確認
df_result_3m = df_result_3m.groupby(["customercd","coupon_name"]).head(1)


# 4月の結果
df_date2 = pd.DataFrame()
for i in df_cupon_use_kyotu.query("salesday>=20230401&salesday<=20230430")["customer_id"].unique():
    # 日付を作る
    df_date = pd.DataFrame({"salesday":pd.date_range(start="2023-04-01",end="2023-04-30")})
    df_date["a"] = 1
    # クーポン対象商品を出す
    df_a = df_cupon_use_kyotu.query("customer_id==@i").reset_index().drop("index",axis=1)[["salesday","coupon_name","jan2","customer_id"]]\
    .rename(columns={"salesday":"start_date"})
    df_a["a"] = 1
    # 日毎に作る
    df_date = df_date.merge(df_a,on="a")
    del df_date["a"]
    df_date["flg"] = df_date.apply(lambda x:1 if x["start_date"]<=x["salesday"] else 0, axis=1)
    df_date = df_date.query("flg==1")
    df_date2 = pd.concat([df_date2,df_date])

df_result_4m = df_jan.merge(df_date2.rename(columns={"customer_id":"customercd"}),on=["salesday","customercd","jan2"])

# 重複がないかは確認
df_result_4m = df_result_4m.groupby(["customercd","coupon_name"]).head(1)

# 重複を削除
df_cupon_use = df_cupon_use.groupby(["coupon_name","customer_id"]).head(1)
df_cupon_use_kyotu = df_cupon_use_kyotu.groupby(["coupon_name","customer_id"]).head(1)




# 3月の結果について（PBのみ）
# 利用者
df_tmp_riyou_3m = df_result_3m.groupby(["salesday"])\
.agg({"flg":"sum","customercd":"nunique"}).reset_index().rename(columns={"flg":"クーポン利用人数","customercd":"クーポン利用人数UU"})
# 宣言人数
df_tmp_sengen_3m = df_cupon_use_kyotu.query("salesday>=20230306&salesday<=20230331").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"宣言人数","customer_id":"宣言人数UU"})
# クーポンタブ閲覧
df_tmp_eturan_3m = df_c_view.query("salesday>=20230306&salesday<=20230331").merge(df_hm,on="customer_id").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"クーポンタブ閲覧人数","customer_id":"クーポンタブ閲覧UU"})

# 累積宣言UU
df_sengen_rui_3m = df_cupon_use_kyotu.sort_values("salesday").query("salesday>=20230306&salesday<=20230331")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積宣言UU"})
df_sengen_rui_3m["累積宣言UU"]= df_sengen_rui_3m["累積宣言UU"].cumsum()

# 累積クーポンタブUU
df_eturan_rui_3m = df_c_view.merge(df_hm,on="customer_id").sort_values("salesday").query("salesday>=20230306&salesday<=20230331")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積クーポン閲覧UU"})
df_eturan_rui_3m["累積クーポン閲覧UU"]= df_eturan_rui_3m["累積クーポン閲覧UU"].cumsum()

# 累積購入クーポンUU
df_riyo_rui3m = df_result_3m.sort_values("salesday").query("salesday>=20230306&salesday<=20230331")\
.groupby(["customercd"]).head(1).groupby(["salesday"]).agg({"customercd":"count"}).reset_index().rename(columns={"customercd":"累積クーポン利用UU"})
df_riyo_rui3m["累積クーポン利用UU"]= df_riyo_rui3m["累積クーポン利用UU"].cumsum()

home_3m_kyoutu = df_home_view.query("salesday>=20230306&salesday<=20230331").merge(df_hm,on="customer_id")["customer_id"].nunique()

# 結果を表示（共通のみ）
df_result2 = df_tmp_riyou_3m.merge(df_tmp_sengen_3m,on="salesday",how="outer")
df_result2 = df_result2.merge(df_tmp_eturan_3m, on="salesday",how="outer")
df_result2 = df_result2.merge(df_sengen_rui_3m, on="salesday",how="outer")
df_result2 = df_result2.merge(df_eturan_rui_3m, on="salesday",how="outer")
df_result2 = df_result2.merge(df_riyo_rui3m, on="salesday",how="outer")
df_result2 = df_result2.rename(columns={"salesday":"日付"})
df_result2["アプリ起動UU"] = home_3m_kyoutu

df_result2.query("日付=='20230331'")[["アプリ起動UU","累積クーポン閲覧UU","累積宣言UU","累積クーポン利用UU"]]

,アプリ起動UU,累積クーポン閲覧UU,累積宣言UU,累積クーポン利用UU
24,2183,1114,235,144.0


In [348]:
# 4月の結果について（共通のみ）
# 利用者
df_tmp_riyou_4m = df_result_4m.groupby(["salesday"])\
.agg({"flg":"sum","customercd":"nunique"}).reset_index().rename(columns={"flg":"クーポン利用人数","customercd":"クーポン利用人数UU"})
# 宣言人数
df_tmp_sengen_3m = df_cupon_use_kyotu.query("salesday>=20230401&salesday<=20230430").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"宣言人数","customer_id":"宣言人数UU"})
# クーポンタブ閲覧
df_tmp_eturan_3m = df_c_view.query("salesday>=20230401&salesday<=20230430").merge(df_hm,on="customer_id").groupby(["salesday"]).agg({"sync_dttm":"count","customer_id":"nunique"})\
.reset_index().rename(columns={"sync_dttm":"クーポンタブ閲覧人数","customer_id":"クーポンタブ閲覧UU"})

# 累積宣言UU
df_sengen_rui_3m = df_cupon_use_kyotu.sort_values("salesday").query("salesday>=20230401&salesday<=20230430")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積宣言UU"})
df_sengen_rui_3m["累積宣言UU"]= df_sengen_rui_3m["累積宣言UU"].cumsum()

# 累積クーポンタブUU
df_eturan_rui_3m = df_c_view.merge(df_hm,on="customer_id").sort_values("salesday").query("salesday>=20230401&salesday<=20230430")\
.groupby(["customer_id"]).head(1).groupby(["salesday"]).agg({"customer_id":"count"}).reset_index().rename(columns={"customer_id":"累積クーポン閲覧UU"})
df_eturan_rui_3m["累積クーポン閲覧UU"]= df_eturan_rui_3m["累積クーポン閲覧UU"].cumsum()

# 累積購入クーポンUU
df_riyo_rui3m = df_result_4m.sort_values("salesday").query("salesday>=20230401&salesday<=20230430")\
.groupby(["customercd"]).head(1).groupby(["salesday"]).agg({"customercd":"count"}).reset_index().rename(columns={"customercd":"累積クーポン利用UU"})
df_riyo_rui3m["累積クーポン利用UU"]= df_riyo_rui3m["累積クーポン利用UU"].cumsum()

home_4m_kyoutu = df_home_view.merge(df_hm,on="customer_id").query("salesday>=20230401&salesday<=20230430")["customer_id"].nunique()

# 結果を表示（PBのみ）
df_result2_4m = df_tmp_riyou_3m.merge(df_tmp_sengen_3m,on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_tmp_eturan_3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_sengen_rui_3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_eturan_rui_3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.merge(df_riyo_rui3m, on="salesday",how="outer")
df_result2_4m = df_result2_4m.rename(columns={"salesday":"日付"})
df_result2_4m["アプリ起動UU"] = home_4m_kyoutu

df_result2_4m.query("日付=='20230430'")[["アプリ起動UU","累積クーポン閲覧UU","累積宣言UU","累積クーポン利用UU"]]

,アプリ起動UU,累積クーポン閲覧UU,累積宣言UU,累積クーポン利用UU
54,2183,995.0,198.0,131.0


In [349]:
df_hm["customer_id"].nunique()

2265

## 結果について

In [350]:

# 共通クーポンのみに絞る
df_taishoh_id_kyoutu = df_taisho_id[["coupon_name","kubun"]].drop_duplicates()
df_cupon_use_kyotu = df_cupon_use.merge(df_taishoh_id_kyoutu,on="coupon_name")


# 3月の結果 df_cupon_use
df_date2 = pd.DataFrame()
for i in df_cupon_use_kyotu.query("salesday>=20230306&salesday<=20230331")["customer_id"].unique():
    # 日付を作る
    df_date = pd.DataFrame({"salesday":pd.date_range(start="2023-03-06",end="2023-03-31")})
    df_date["a"] = 1
    # クーポン対象商品を出す
    df_a = df_cupon_use_kyotu.query("customer_id==@i").reset_index().drop("index",axis=1)[["salesday","coupon_name","jan2","customer_id"]]\
    .rename(columns={"salesday":"start_date"})
    df_a["a"] = 1
    # 日毎に作る
    df_date = df_date.merge(df_a,on="a")
    del df_date["a"]
    df_date["flg"] = df_date.apply(lambda x:1 if x["start_date"]<=x["salesday"] else 0, axis=1)
    df_date = df_date.query("flg==1")
    df_date2 = pd.concat([df_date2,df_date])

df_result_3m = df_jan.merge(df_date2.rename(columns={"customer_id":"customercd"}),on=["salesday","customercd","jan2"])
# df_result_3m = df_jan.merge(df_date2.rename(columns={"jan":"jan2","user_id":"customercd2"}),on=["salesday","customercd2","jan2"])
# 重複がないかは確認
df_result_3m = df_result_3m.groupby(["customercd","coupon_name"]).head(1)


# 4月の結果
df_date2 = pd.DataFrame()
for i in df_cupon_use_kyotu.query("salesday>=20230401&salesday<=20230430")["customer_id"].unique():
    # 日付を作る
    df_date = pd.DataFrame({"salesday":pd.date_range(start="2023-04-01",end="2023-04-30")})
    df_date["a"] = 1
    # クーポン対象商品を出す
    df_a = df_cupon_use_kyotu.query("salesday>=20230401&salesday<=20230430")\
    .query("customer_id==@i").reset_index().drop("index",axis=1)[["salesday","coupon_name","jan2","customer_id"]]\
    .rename(columns={"salesday":"start_date"})
    df_a["a"] = 1
    # 日毎に作る
    df_date = df_date.merge(df_a,on="a")
    del df_date["a"]
    df_date["flg"] = df_date.apply(lambda x:1 if x["start_date"]<=x["salesday"] else 0, axis=1)
    df_date = df_date.query("flg==1")
    df_date2 = pd.concat([df_date2,df_date])

df_result_4m = df_jan.merge(df_date2.rename(columns={"customer_id":"customercd"}),on=["salesday","customercd","jan2"])

# 重複がないかは確認
df_result_4m = df_result_4m.groupby(["customercd","coupon_name"]).head(1)

In [420]:
# df_date2
# df_result_4m.groupby(["customercd","coupon_name"]).head(1).query("coupon_name=='おかめ仕立てうまかたれ付45g×3'").sort_values("salesday")["customercd"].nunique()
df_date2.query("coupon_name=='おかめ仕立てうまかたれ付45g×3'")["customer_id"].nunique()

201

In [424]:
df_cupon_use.query("coupon_name=='おかめ仕立てうまかたれ付45g×3'")

,dt,sync_dttm,customer_id,coupon_id,coupon_name,salesday,jan,jan2
1857,2023-04-01,2023-04-01 14:47:01.451,e1a4fcf4a8d55c29493b06b70155d644,95,おかめ仕立てうまかたれ付45g×3,2023-04-01,4901160010174,4901160010174
1858,2023-04-01,2023-04-01 09:38:36.315,78a0056ddb292527af0651c44b0b4033,95,おかめ仕立てうまかたれ付45g×3,2023-04-01,4901160010174,4901160010174
1859,2023-04-01,2023-04-01 17:29:54.426,862212a64ebf3f61f77ab5dcf10f8edb,95,おかめ仕立てうまかたれ付45g×3,2023-04-01,4901160010174,4901160010174
1860,2023-04-01,2023-04-01 06:01:25.023,e24a0d690e77086ecd3c9833b830d165,95,おかめ仕立てうまかたれ付45g×3,2023-04-01,4901160010174,4901160010174
1861,2023-04-01,2023-04-01 03:14:22.525,352d6a6ac00742455f4375494b715bff,95,おかめ仕立てうまかたれ付45g×3,2023-04-01,4901160010174,4901160010174
...,...,...,...,...,...,...,...,...
3585,2023-04-30,2023-04-30 07:03:35.501,12e4afeb9b31b3cfc3d704196b4a9c00,95,おかめ仕立てうまかたれ付45g×3,2023-04-30,4901160010174,4901160010174
3586,2023-04-30,2023-04-30 23:47:52.501,72134072235bf6a4af3e8500cc3945c8,95,おかめ仕立てうまかたれ付45g×3,2023-04-30,4901160010174,4901160010174
3587,2023-04-30,2023-04-30 10:41:17.569,c1a42e2f8a616ac8896986293cce598a,95,おかめ仕立てうまかたれ付45g×3,2023-04-30,4901160010174,4901160010174
3588,2023-04-30,2023-04-30 17:49:39.430,0281802a71330d9edd879a7a3ddefe7a,95,おかめ仕立てうまかたれ付45g×3,2023-04-30,4901160010174,4901160010174


In [351]:
# query = """
#   SELECT *
#   FROM `inunaki.dst_workspace.app_cupon_jan`
#   WHERE haisin_tuki BETWEEN "2023-03-01" AND "2023-05-30"
# """
# df_cp_jan = client.query(query).to_dataframe()

In [400]:
df_cp_jan = df_taisho_id.copy()
df_cp_jan["jan"] = df_cp_jan["jan"].astype(int)

In [401]:
# ユニークのみにする
df_cp_jan2 = df_cp_jan[["jan","kubun","haisin_tuki"]].drop_duplicates()

# 利用につながった人数
df_cp_3m = df_result_3m.drop("jan",axis=1).rename(columns={"jan2":"jan"}).merge(df_cp_jan2[["jan","kubun"]],on="jan")
df_cp_4m = df_result_4m.drop("jan",axis=1).rename(columns={"jan2":"jan"}).merge(df_cp_jan2[["jan","kubun"]],on="jan")
 # 利用した人数の集計
df_cp_3m = df_cp_3m.groupby(["coupon_name","kubun"]).count().reset_index()[["coupon_name","kubun","customercd"]].rename(columns={"customercd":"利用人数"})
df_cp_4m = df_cp_4m.groupby(["coupon_name","kubun"]).count().reset_index()[["coupon_name","kubun","customercd"]].rename(columns={"customercd":"利用人数"})

In [402]:
# 宣言した人数
df_cp_use_m = df_cupon_use.drop("jan",axis=1).rename(columns={"jan2":"jan"}).merge(df_cp_jan2[["jan","kubun","haisin_tuki"]],on="jan")
df_cp_use_m["haisin_tuki"] = pd.to_datetime(df_cp_use_m["haisin_tuki"])
df_cp_use_3m = df_cp_use_m.query("haisin_tuki==20230301")
df_cp_use_4m = df_cp_use_m.query("haisin_tuki==20230401")

# 宣言人数の集計
df_cp_use_3m = df_cp_use_3m.groupby(["coupon_name","kubun"]).count().reset_index()[["coupon_name","kubun","customer_id"]].rename(columns={"customer_id":"宣言人数"})
df_cp_use_4m = df_cp_use_4m.groupby(["coupon_name","kubun"]).count().reset_index()[["coupon_name","kubun","customer_id"]].rename(columns={"customer_id":"宣言人数"})

In [403]:
# 閲覧人数
df_c_view_m = df_c_view.merge(df_taisho_id_rank,on="customer_id")
df_c_view_m_all = df_c_view_m.copy()
df_c_view_m_pb = df_c_view_m.query("ランク=='H'|ランク=='M'")

# クーポン閲覧人数3月
df_c_view_3m_all = df_c_view_m_all.query("salesday>=20230306&salesday<=20230331")
df_c_view_3m_all = pd.DataFrame({"kubun":["共通"],"閲覧UU":[df_c_view_3m_all["customer_id"].nunique()]})
df_c_view_3m_pb = df_c_view_m_pb.query("salesday>=20230306&salesday<=20230331")
df_c_view_3m_pb = pd.DataFrame({"kubun":["PB"],"閲覧UU":[df_c_view_3m_pb["customer_id"].nunique()]})
# クーポン閲覧人数4月
df_c_view_4m_all = df_c_view_m_all.query("salesday>=20230401&salesday<=20230430")
df_c_view_4m_all = pd.DataFrame({"kubun":["共通"],"閲覧UU":[df_c_view_4m_all["customer_id"].nunique()]})
df_c_view_4m_pb = df_c_view_m_pb.query("salesday>=20230401&salesday<=20230430")
df_c_view_4m_pb = pd.DataFrame({"kubun":["PB"],"閲覧UU":[df_c_view_4m_pb["customer_id"].nunique()]})
# 月毎にまとめる
df_3m_view = pd.concat([df_c_view_3m_all,df_c_view_3m_pb])
df_4m_view = pd.concat([df_c_view_4m_all,df_c_view_4m_pb])

In [404]:
# アクティブ人数 
df_home_view_m = df_home_view.merge(df_taisho_id_rank,on="customer_id")
df_home_view_m_all = df_home_view_m.copy()
df_home_view_m_pb = df_home_view_m.query("ランク=='H'|ランク=='M'")

# クーポン閲覧人数3月
df_home_view_3m_all = df_home_view_m_all.query("salesday>=20230306&salesday<=20230331")
df_home_view_3m_all = pd.DataFrame({"kubun":["共通"],"アクティブユーザー":[df_home_view_3m_all["customer_id"].nunique()]})
df_home_view_3m_pb = df_home_view_m_pb.query("salesday>=20230306&salesday<=20230331")
df_home_view_3m_pb = pd.DataFrame({"kubun":["PB"],"アクティブユーザー":[df_home_view_3m_pb["customer_id"].nunique()]})
# クーポン閲覧人数4月
df_home_view_4m_all = df_home_view_m_all.query("salesday>=20230401&salesday<=20230430")
df_home_view_4m_all = pd.DataFrame({"kubun":["共通"],"アクティブユーザー":[df_home_view_4m_all["customer_id"].nunique()]})
df_home_view_4m_pb = df_home_view_m_pb.query("salesday>=20230401&salesday<=20230430")
df_home_view_4m_pb = pd.DataFrame({"kubun":["PB"],"アクティブユーザー":[df_home_view_4m_pb["customer_id"].nunique()]})

# 月毎にまとめる
df_3m_home_view = pd.concat([df_home_view_3m_all,df_home_view_3m_pb])
df_4m_home_view = pd.concat([df_home_view_4m_all,df_home_view_4m_pb])

In [428]:
df_3_result = df_cp_3m.merge(df_cp_use_3m,on=["coupon_name","kubun"],how="outer")
df_3_result = df_3_result.merge(df_3m_view,on="kubun").merge(df_3m_home_view,on="kubun")
df_3_result = df_3_result[["coupon_name","kubun","アクティブユーザー","閲覧UU","宣言人数","利用人数"]].rename(columns={"coupon_name":"クーポン名","kubun":"区分"})
# 結果を出す
df_4_result = df_cp_4m.merge(df_cp_use_4m,on=["coupon_name","kubun"],how="outer")
df_4_result = df_4_result.merge(df_4m_view,on="kubun").merge(df_4m_home_view,on="kubun")
df_4_result = df_4_result[["coupon_name","kubun","アクティブユーザー","閲覧UU","宣言人数","利用人数"]].rename(columns={"coupon_name":"クーポン名","kubun":"区分"})
# df_4_result = df_4_result.dropna()

In [430]:
df_4_result.to_csv("4月の配信.csv",index=False,encoding="utf_8_sig")

df_3_result.sort_values("利用人数",ascending=False).to_csv("3月の配信.csv",index=False,encoding="utf_8_sig")

In [431]:
df_taisho_id_rank

,customer_id,customercd,ランク
0,fdc455269e47f3ae13733e6a559b34d1,2970000472993,E
1,ff9d15ad67b99edf58ef72c7a9419c9c,2970001262265,E
2,febb5849be0f51acb5f51137e48410b4,2970002751386,L
3,fff4176e3e847a8c206da0c9f85a29be,2970001400964,E
4,fdcfd848653f1da4eed0defb12584ce2,2970000197674,L
...,...,...,...
9623,997345e782575a4b6beca3dfde4d1915,2970001602948,L
9624,97790fce418e33e1ae2b8add8e73d7e6,2970000956028,E
9625,97bffdb3710d16b01d4f91884ab49da2,2970002890443,M
9626,982bf8e93695231dbb6975fe4d63da12,2970001055089,E


In [433]:
df_hm

,customer_id
5,fff6b260a5368608567f2dcd2465dab4
6,ff860e4466aba37c053854a86518032f
9,fde82cdcbadd5139c0be894e6083f02a
17,ff8aebed598886507bd6a270298a6073
22,fef65bd874792877de29b841f8fbf456
...,...
9612,97e29e2ff8f46eb67def0dbbc51c4599
9614,99fdca819b2ff5160b0f0390e728dd4a
9617,97f9a850b363d694962d774f96d66ec5
9622,96787014ca10ce60fce6cb2ae91ae6cf
